<a href="https://colab.research.google.com/github/SrikarSai99/NNDL/blob/main/NNDL_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import math as mt
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data=pd.read_csv('/content/drive/My Drive/NNDL Datasets/Load Dataset.csv')
data.head()

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
x = []
y = []
for i in range(1, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24]) 
    y.append(data['Load (kW)'].iloc[i])

In [5]:
data = pd.DataFrame({'Previous Hour Load' : x, 'Present Hour Load' : y})
data.head()

,Previous Hour Load,Present Hour Load
0,4965.86916,4983.17184
1,4783.95720,4888.39680
2,4663.92960,5072.95872
3,4080.93840,5196.25980
4,4355.28720,5641.29720


In [6]:
maxX = np.max(data['Previous Hour Load'])
minX = np.min(data['Previous Hour Load'])
maxY = np.max(data['Present Hour Load'])
minY = np.min(data['Present Hour Load'])

data['Previous Hour Load'] = (data['Previous Hour Load'] - minX) / (maxX - minX)
data['Present Hour Load'] = (data['Present Hour Load'] - minY) / (maxY - minY)

In [7]:
x = data['Previous Hour Load']
y = data['Present Hour Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [8]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.1

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        tempM = m + v*vm
        tempC = c + v*vc
        gradM = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC)
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        m += vm
        c += vc
print("Value of m = ",m)
print("Value of c = ",c)

Value of m =  0.7310975534736028
Value of c =  0.13135815340132778


In [9]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

In [10]:
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error Values :')
print(f'MAE = {training_MAE}')
print(f'MSE = {training_MSE}')
print(f'RMSE = {training_RMSE}')

Training Error Values :
MAE = 527.809963213077
MSE = 512781.53025965503
RMSE = 716.0876554302937


In [11]:
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error Values :')
print(f'MAE = {testing_MAE}')
print(f'MSE = {testing_MSE}')
print(f'RMSE = {testing_RMSE}')

Testing Error Values :
MAE = 593.0800930470241
MSE = 604069.0723003822
RMSE = 777.2188059358717


In [13]:
x = float(input('Enter Load at Previous Hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted Load value at Present Hour = ', prediction)

Enter Load at Previous Hour : 3000
Predicted Load value at Present Hour =  3819.331599913804
